<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Walmart_logo.svg/1280px-Walmart_logo.svg.png" height="100px">

Ce projet de machine learning supervisé pour Walmart vise à construire un modèle capable d'estimer les ventes hebdomadaires dans leurs magasins avec la meilleure précision possible. Voici un rappel simple des objectifs du projet :

1. **Exploration de données (Partie 1) :** Effectuer une analyse exploratoire des données (EDA) et prétraiter les données pour les préparer à l'apprentissage automatique.
   
2. **Modèle de régression linéaire de base (Partie 2) :** Entraîner un modèle de régression linéaire simple pour prédire le montant des ventes hebdomadaires en fonction des autres variables. Évaluer les performances du modèle et interpréter les coefficients pour identifier les caractéristiques importantes pour la prédiction.

3. **Éviter le surajustement (Partie 3) :** Entraîner un modèle de régression linéaire régularisé (Ridge ou Lasso) pour réduire le surajustement. Fine-tuning des hyperparamètres pour obtenir les meilleures prédictions généralisées.

# Import Lib

In [356]:
import pandas as pd 
import sys
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,mean_squared_error
import warnings
from sklearn.linear_model import LinearRegression
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso,Ridge
from sklearn.model_selection import GridSearchCV, cross_val_score

warnings.filterwarnings(
    "ignore", category=DeprecationWarning
)  # to avoid deprecation warnings

import sys
sys.path.append(r'C:\Users\antoi\Documents\Work_Learn\JEDHA\M05-Supervised_ML\JEDHA-Projet')
from function import *

# EDA

In [357]:
df = pd.read_csv("Walmart_Store_sales.csv")
describe_df(df)

number of rows: 150
Display of dataset:


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


Basic statistics:


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


Pourcentage of missing values:


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

`Explications du jeu de donnée`

-------

- `Store`: Identifiant du magasin (20 magasins différents).
- `Date`: Date des ventes (entre 2010 et 2012).
- `Weekly_Sales`: Ventes hebdomadaires.
- `Holiday_Flag`: Indicateur binaire (0 ou 1) indiquant si la semaine correspondante est une semaine de vacances.
- `Temperature`: Température pendant la semaine.
- `Fuel_Price`: Prix du carburant pendant la semaine.
- `CPI`: Indice des prix à la consommation (CPI).
- `Unemployment`: Taux de chômage.

Le jeu de données comporte initialement 150 lignes, avec des valeurs manquantes dans toutes les colonnes (excepté pour Store). Nous allons devoir procéder à un nettoyage des données, puisqu'il nous manque 9% des données de notre variable cible (WeeklySales). Nous allons séparer notre colonne Date en années, mois et jours (de l'année et de la semaine). Enfin, nous allons enlever les valeurs aberrantes (au-delà de mean ± 3σ).

Après ce prétraitement, il ne nous reste plus que 117 lignes pour effectuer notre étude. De plus, on peut se rendre compte que les jours de la semaine ne seront pas utiles car les valeurs sont identiques (nous aurions pu nous en douter car notre variable cible se focalise sur les ventes hebdomadaires). Les jours de l'année sont redondants avec l'information mensuelle.

In [358]:
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day_of_week'] = df['Date'].dt.day_of_week
df['day_of_year'] = df['Date'].dt.day_of_year

df = df.drop(['Date'],axis=1) 

C:\Users\antoi\AppData\Local\Temp\ipykernel_16872\1572134870.py:1: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [359]:
print('{} samples before'.format(df.shape[0]))

df.dropna(subset=['Weekly_Sales'],inplace=True)
df = df.reset_index(drop=True)

cols = ['Temperature', 'Fuel_Price', 'CPI','Unemployment']

for col in cols:
    col_mean = df[col].mean()
    col_std = df[col].std()
    
    lower_bound = col_mean - 3 * col_std
    upper_bound = col_mean + 3 * col_std
    df_filtered = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    outliers = df[(df[col] > upper_bound) | (df[col] < lower_bound)]
    if not outliers.empty:
        print(f"Outliers in column '{col}': {len(outliers)} rows")

df = df_filtered
print('{} samples after'.format(df.shape[0]))


150 samples before
Outliers in column 'Unemployment': 5 rows
117 samples after


### Plot

**Obersvation du jeu de donnée**

On observe une grandre disparité de vente entre les magasins (il)

In [360]:
fig = px.histogram(df,x="Unemployment",y="CPI",color= df["year"].astype(str).tolist(),nbins=20)
fig.update_layout(bargap=0.1, xaxis=dict(tickmode='linear', dtick=1))
fig.show()

In [361]:
fig = px.histogram(df,x="Store",y="Weekly_Sales",color= df["year"].astype(str).tolist(),nbins=20)
fig.update_layout(bargap=0.1, xaxis=dict(tickmode='linear', dtick=1))
fig.show()

In [362]:
fig = px.histogram(df,x="month",y="Weekly_Sales",color= df["year"].astype(str).tolist(),nbins=12)
fig.update_layout(bargap=0.1)

fig.show()

In [363]:


import plotly.express as px
fig = px.histogram(df, x="year", nbins=len(df['year'].unique()))
fig.update_layout(bargap=0.1)
fig.show()


fig = px.histogram(df, x="month", nbins=12)
fig.update_layout(bargap=0.1)
fig.show()


fig = px.histogram(df, x="CPI",nbins=100)
fig.update_layout(bargap=0.1)
fig.show()

**Qu'observe-t-on sur ces graphes ?**

In [364]:
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=1000, width = 1000)
fig.show()

Nous n'observons pas de tendance ou de relation lineaire (intéressante) à première vue. Intéressante, car il y a des lien évidents entre le mois et la température par exemple.

In [365]:
corr_matrix = df.corr().round(2)
fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist()
                                  )

fig.update_layout(height=800, width = 800)

fig.show()

Même remarque que pour le graphe précédent, ici de manière plus parlante. L'augmentation du prix de l'essence au cours des années, la température au fil des mois ne nous intéresse pas. Il est intéressant de s'intéresser à la corrélation entre la target et le reste des features. La features qui semblerait la plus corréler au Weekly_Sales serait le CPI.

# First Model

Nous allons faire un premier modèle de prédiction des Weekly Sales en fonction des features initiales afin d'avoir un premier modèle de référence. Nous distinguons les variables numériques et catégorielles. Comprenant dans ce premier modèle:
- target: Weekly Sales
- numerique: Fuel_Price, Temperature, CPI, Unemployment, year, month,Store
- catégorielle: _


Etant donnée que nous avons des valeurs manquantes, il est nécessaire de remplacer ces valeurs. Pour les numériques nous utilisons une première stratégie de base, on remplace par la moyenne. Pour les catégorielles, par la valeurs la plus fréquente.

Nous mettons en place un jeu de test représentant 20% des données, avec un modèle de regression linéaire par défaut de la librairie sklearn sans jouer sur les hyperparamètres dans un premier temps.

In [366]:
target = 'Weekly_Sales'
# remove_col = ['day_of_week','day_of_year', 'Holiday_Flag']
remove_col = ['day_of_week','day_of_year']

df.drop(remove_col,axis=1,inplace=True)

In [367]:
numeric_strategy="median"
categorical_strategy="most_frequent"

In [368]:
numeric_features = ['CPI']

Y = df[target]
X = df[numeric_features]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=numeric_strategy)),
        ("scaler", StandardScaler()),
    ]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
    ]
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test) 

model = LinearRegression()
model.fit(X_train, Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print("Accuracy on training set : ", model.score(X_train, Y_train))
print("Accuracy on test set : ", model.score(X_test, Y_test))


# Print the mean and standard deviation of the scores
scores = cross_val_score(model, X_train, Y_train, cv=3)
print("Mean cross-validation score:", scores.mean())
print("Standard deviation of cross-validation scores:", scores.std())


Accuracy on training set :  0.09419825900762302
Accuracy on test set :  0.14226114407145063
Mean cross-validation score: 0.06451411105790035
Standard deviation of cross-validation scores: 0.07434324859447619


Conclusions:

Comme observé sur le graphe de corrélation, l'indice le plus réprésentatif de nos features est le CPI. Cependant l'accuracy du train modèle est très faible (0.14), de plus le modèle semble overfitter . Ce qui peut paraitre normal pour un premier modèle c'est pourquoi nous allons affuter notre modèle et le choix des features.

# Second model

En réalité, il n'y a pas de sens a considéré la colonnes `Store`, `HolidaysFlag`, `year`et `month` comme des variables numériques.

In [369]:
categorical_features = ['Store','Holiday_Flag']
numeric_features = [col for col in df.columns if col not in (categorical_features and target)]

Y = df[target]
X = df.drop(target,axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=numeric_strategy)),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=categorical_strategy)),
        ("encoder", OneHotEncoder(drop="first")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test) 

model = LinearRegression()
model.fit(X_train, Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print("Accuracy on training set : ", model.score(X_train, Y_train))
print("Accuracy on test set : ", model.score(X_test, Y_test))


# Print the mean and standard deviation of the scores
scores = cross_val_score(model, X_train, Y_train, cv=3)
print("Mean cross-validation score:", scores.mean())
print("Standard deviation of cross-validation scores:", scores.std())

coefs = pd.DataFrame(index = preprocessor.get_feature_names_out(), data = model.coef_.transpose(), columns=["coefficients"])
feature_importance = abs(coefs).sort_values(by = 'coefficients',ascending=False)

fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )

fig.show()

Accuracy on training set :  0.9687405020385081
Accuracy on test set :  0.9498641158138467
Mean cross-validation score: 0.839730912294138
Standard deviation of cross-validation scores: 0.0759288886042718


Les éléments qui semblent être à l'origine des prédictions sont principalement les identifiants des magasins (Store). Bien que nous obtenions un score nettement plus élevé qu'auparavant, il semble encore y avoir un léger surapprentissage. Nous pouvons faire l'hypothèse qu'aucune des colonnes à part certains stores sont capables de prédire les ventes heabdomadaire. Nous pouvons essayer de retirer ces colonnes. 

Ces résultats indiquent que le modèle fonctionne bien sur les données d'entraînement et de test, mais il peut encore être amélioré pour réduire le surapprentissage.

## Only `Store` category

In [370]:
categorical_features = ['Store']

Y = df[target]
X = df[categorical_features]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=categorical_strategy)),
        ("encoder", OneHotEncoder(drop="first")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
    ]
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test) 

model = LinearRegression()
model.fit(X_train, Y_train)
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print("Accuracy on training set : ", model.score(X_train, Y_train))
print("Accuracy on test set : ", model.score(X_test, Y_test))

# Print the mean and standard deviation of the scores
scores = cross_val_score(model, X_train, Y_train, cv=3)
print("Mean cross-validation score:", scores.mean())
print("Standard deviation of cross-validation scores:", scores.std())


coefs = pd.DataFrame(index = preprocessor.get_feature_names_out(), data = model.coef_.transpose(), columns=["coefficients"])
feature_importance = abs(coefs).sort_values(by = 'coefficients',ascending=False)

fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )

fig.show()

Accuracy on training set :  0.9529586495347379
Accuracy on test set :  0.9619161375649664
Mean cross-validation score: 0.8579259052595863
Standard deviation of cross-validation scores: 0.05279193538610378


Conclusion :

En éliminant les colonnes qui n'apportent pas d'informations, nous avons réussi à améliorer le `cross validation score` ainsi que le score sur l'ensemble d'entraînement. Cependant, il reste encore à déterminer si nous pouvons optimiser davantage le modèle, notamment en explorant les meilleurs hyperparamètres à l'aide d'une `Grid Search` à l'aide de `Ridge` ou `Lasso`. De plus, nous ajouterons un `max_iter` qui permet de continuer les itérations si l'on ne converge pas.

# LASSO

In [372]:
lasso = Lasso(max_iter=10000)

params = {
    'alpha': np.arange(1,1000)
}

grid_search = GridSearchCV(lasso, params, cv=5)
grid_search.fit(X_train, Y_train)

best_alpha = grid_search.best_params_['alpha']
best_lasso = grid_search.best_estimator_

best_lasso.fit(X_train, Y_train)

Y_train_pred = best_lasso.predict(X_train)
Y_test_pred = best_lasso.predict(X_test)

train_score = best_lasso.score(X_train, Y_train)
test_score = best_lasso.score(X_test, Y_test)
print("Best alpha:", best_alpha)
print("Accuracy on training set:", train_score)
print("Accuracy on test set:", test_score)


scores = cross_val_score(best_lasso, X_train, Y_train, cv=5)

# Print the mean and standard deviation of the scores
print("Mean cross-validation score:", scores.mean())
print("Standard deviation of cross-validation scores:", scores.std())

Best alpha: 744
Accuracy on training set: 0.9522965394680727
Accuracy on test set: 0.9582901959768352
Mean cross-validation score: 0.9216910122169251
Standard deviation of cross-validation scores: 0.045774498598366485


Nous remarquons que le `cross-validation score` s'est encore amélioré au vu de notre recherche de meilleur hyperparamètre `alpha` selon la méthode Lasso.

In [ ]:
ridge = Ridge(max_iter=10000)

params = {
    'alpha': np.arange(0.01,10,0.01)
}

grid_search = GridSearchCV(ridge, params, cv=5)
grid_search.fit(X_train, Y_train)

best_alpha = grid_search.best_params_['alpha']
best_ridge = grid_search.best_estimator_

best_ridge.fit(X_train, Y_train)

Y_train_pred = best_ridge.predict(X_train)
Y_test_pred = best_ridge.predict(X_test)

train_score = best_ridge.score(X_train, Y_train)
test_score = best_ridge.score(X_test, Y_test)
scores = cross_val_score(best_ridge, X_train, Y_train, cv=5)


print("Best alpha:", best_alpha)
print("Accuracy on training set:", train_score)
print("Accuracy on test set:", test_score)
print("Mean cross-validation score:", scores.mean())
print("Standard deviation of cross-validation scores:", scores.std())

Best alpha: 0.02
Accuracy on training set: 0.9528895277114315
Accuracy on test set: 0.960363279386515
Mean cross-validation score: 0.9200559500858626
Standard deviation of cross-validation scores: 0.04575632232979749


Comme pour la méthode Lasso, le Ridge nous retourne un `cross-validation score` meilleur que ceux des modèles précédents, même s'il semble légèrement inférieur au Lasso.

# Conclusion Walmart

<br>

### Résumé

Ce projet de machine learning supervisé pour Walmart visait à construire un modèle capable d'estimer les ventes hebdomadaires dans leurs magasins avec la meilleure précision possible. Voici un rappel simple des objectifs du projet :

1. `Exploration de données (Partie 1) :` Effectuer une analyse exploratoire des données (EDA) et prétraiter les données pour les préparer à l'apprentissage automatique.
   
2. `Modèle de régression linéaire de base (Partie 2) :` Entraîner un modèle de régression linéaire simple pour prédire le montant des ventes hebdomadaires en fonction des autres variables. Évaluer les performances du modèle et interpréter les coefficients pour identifier les caractéristiques importantes pour la prédiction.

3. `Éviter le surajustement (Partie 3) :` Entraîner un modèle de régression linéaire régularisé (Ridge ou Lasso) pour réduire le surajustement. Fine-tuning des hyperparamètres pour obtenir les meilleures prédictions généralisées.

En résumé, le projet comprend l'exploration des données, la construction d'un modèle de régression linéaire de base, et la mise en œuvre de techniques de régularisation pour améliorer la généralisation du modèle.


<br>

### Résultats

Le jeu de données était extrêmement petit avec seulement `150 échantillons`, et après prétraitement, `9%` des échantillons de données ont été supprimés. La visualisation de la distribution des données et de leurs relations nous a permis d'obtenir quelques insights sur l'ensemble des fonctionnalités. Tester plusieurs algorithmes avec des hyperparamètres par défaut nous a donné une certaine compréhension des performances des différents modèles sur cet ensemble de données spécifique.

Finalement, le meilleur modèle est un compromis entre la simplicité à l'aide du feature engineering, du choix de colonnes et la complexité de l'amélioration des hyperparamètres. C'est ainsi que le dernier `modèle Lasso` avec uniquement les colonnes Stores permet d'obtenir un score de précision de `0.953` sur le `X_train` et de `0.960` sur le `X_test`, avec un score de `cross-validation de 0.92`.

<br>

### Améliorations

On observe une grande disparité de ventes entre les magasins, il serait intéressant d'avoir davantage d'informations sur leur localisation ou autre pour expliquer ce phénomène, afin d'améliorer le ciblage marketing chez Walmart.

Pour obtenir des informations de meilleure qualité et plus détaillées, il serait judicieux d'explorer davantage de facteurs influençant les ventes. Par exemple, une analyse de la localisation des magasins ou d'autres caractéristiques spécifiques pourrait être extrêmement bénéfique pour expliquer la disparité importante des ventes entre les magasins.

En développant cette idée, une `approche géospatiale` pourrait être utilisée pour examiner les performances de vente en fonction de la localisation des magasins, en tenant compte de facteurs tels que la `densité de population`, le `revenu moyen` dans la région, la `concurrence locale`, et d'autres variables géographiques pertinentes. Cette analyse plus approfondie pourrait fournir des informations précieuses sur les facteurs locaux qui influent sur les ventes et aider à identifier les `opportunités d'amélioration` et de `croissance pour chaque magasin`.
